### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "../Pandas_Challenge/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## 1. Player Count

* Display the total number of players


In [3]:
#number of players
player_number = purchase_data["SN"].nunique()
pn_df = pd.DataFrame({"Total Number of Players": [player_number]})
pn_df

,Total Number of Players
0,576


## 2.Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [67]:
#unique items
unique_items = purchase_data["Item ID"].nunique()
#number of purchases
number_sales = purchase_data["Purchase ID"].count()
#average price
average_price = purchase_data["Price"].sum()/number_sales
#average age
average_age = purchase_data["Age"].sum()/number_sales
#total revenue
total_revenue = purchase_data["Price"].sum()

#summary data frame
summary_info = {"Number of Unique Items": [unique_items],
               "Average Price": [average_price],
               "Number of Purchases": [number_sales],
               "Total Revenue": [total_revenue]
               }



summary_table = pd.DataFrame(summary_info, columns = ["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"])

#formatting
summary_table["Average Price"] = summary_table["Average Price"].map("${:,.2f}".format)
summary_table["Total Revenue"] = summary_table["Total Revenue"].map("${:,.2f}".format)

summary_table

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## 3.Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [70]:
#drop the duplicate SN that occur when the same person places a separate order. 
#This is because we are talking about the count and percentage of people.
refined_sn = purchase_data.drop_duplicates(subset="SN")
gender_countsr = refined_sn["Gender"].value_counts()

total_people = refined_sn["Gender"].count()

gender_as_percentage = (gender_countsr/total_people)

#create new dataframe.
gender_df = pd.DataFrame({ "Total Count": gender_countsr, 
                        "Percentage of Players": gender_as_percentage, 
    })

#formatting
gender_df["Percentage of Players"] = gender_df["Percentage of Players"].map("{:,.2%}".format)
gender_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## 4.Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [81]:
#gender count when including duplicate SNs. 
purchase_count = purchase_data.groupby('Gender')[['Purchase ID']].count()

#total purchase value per gender
total_purchase = purchase_data.groupby('Gender')[['Price']].sum()

df_join = pd.merge(purchase_count, total_purchase, left_index = True, right_index = True, how = 'outer')

#average purchase price
average_price = df_join["Price"]/df_join["Purchase ID"]
average_price1 = total_purchase/purchase_count

average_price
df_join["average_price"] = average_price

#average total price per person
average_total_price = df_join["Price"]/gender_df["Total Count"]

df_join["average_total_price"] = average_total_price

#formatting
df_join.columns = ["Purchase Count", "Total Purchase Value", "Average Purchase Price", "Avg Total Purchase per Person"]
df_join["Average Purchase Price"] = df_join["Average Purchase Price"].map("${:,.2f}".format)
df_join["Avg Total Purchase per Person"] = df_join["Avg Total Purchase per Person"].map("${:,.2f}".format)
df_join["Total Purchase Value"] = df_join["Total Purchase Value"].map("${:,.2f}".format)
df_join

,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,"$1,967.64",$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


## 5.Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [20]:
print(refined_sn["Age"].min())
print(refined_sn["Age"].max())

7
45


In [58]:
#Define bins.
bins = [0, 9, 14, 19, 24, 29, 34, 39, 45 ]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Use values in the age column to assign players to respective bins.
age_df = pd.cut(refined_sn["Age"], bins, labels = group_names)

#Add a new column to the dataframe that specifies age groups of players.
horizontal_stack = pd.concat([refined_sn, age_df], axis = 1)

#Rename columns.
horizontal_stack.columns = ["Purchase ID", "SN", "Age", "Gender", "Item ID", "Item Name", "Price", "Age Group"]

#Calulations
age_group_totals = horizontal_stack["Age Group"].value_counts()

age_group_percentages = age_group_totals/total_people

age_demographics = pd.DataFrame({"Total Count" : age_group_totals, "Percentage of Players": age_group_percentages})

#Formatting
age_demographics["Percentage of Players"] = age_demographics["Percentage of Players"].map("{:,.2%}".format)

#Sort the index.
age_demographics.sort_index()

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## 6.Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [86]:
#Put the purchase data into age bins. 
purchase_data_age = pd.cut(purchase_data["Age"], bins, labels=group_names)

#add the new column into the dataframe.
horizontal_stack_2 = pd.concat([purchase_data, purchase_data_age], axis = 1)

#Two columns have the same name, so rename the binning column 'Age Group'
horizontal_stack_2.columns = ["Purchase ID", "SN", "Age", "Gender", "Item ID", "Item Name", "Price", "Age Group"]

#Find the total purchase value for each age group.
total_purchase_value = horizontal_stack_2.groupby('Age Group')[['Price']].sum()
total_purchase_value.columns = ["Total Purchase Value"]

#Find the purchase count for each age group. 
purchase_count = horizontal_stack_2.groupby('Age Group')[['Purchase ID']].count()
purchase_count.columns = ["Purchase Count"]

#Find the average purchase price for each age group.
average_purchase_price = total_purchase_value["Total Purchase Value"]/purchase_count["Purchase Count"]
average_purchase_price.columns = ["Average Purchase Price"]

#merge total purchase value and purchase count columns into one dataframe.
df_merge1 = pd.merge(total_purchase_value, purchase_count, left_index = True, right_index = True, how = 'outer')

#add average purchase print to the dataframe.
df_merge1["Average Purchase Price"] = average_purchase_price

#Calculation to deduce average total purchase. Add average total purchase to the dataframe. 
average_total_purchase_pp = df_merge1["Total Purchase Value"]/age_demographics["Total Count"]
df_merge1["Average Total Purchase per Person"] = average_total_purchase_pp
df_merge1


#formatting
df_merge1["Total Purchase Value"] = df_merge1["Total Purchase Value"].map("${:,.2f}".format)
df_merge1["Average Purchase Price"] = df_merge1["Average Purchase Price"].map("${:,.2f}".format)
df_merge1["Average Total Purchase per Person"] = df_merge1["Average Total Purchase per Person"].map("${:,.2f}".format)

df_merge1

,Total Purchase Value,Purchase Count,Average Purchase Price,Average Total Purchase per Person
Age Group,,,,
<10,$77.13,23,$3.35,$4.54
10-14,$82.78,28,$2.96,$3.76
15-19,$412.89,136,$3.04,$3.86
20-24,"$1,114.06",365,$3.05,$4.32
25-29,$293.00,101,$2.90,$3.81
30-34,$214.00,73,$2.93,$4.12
35-39,$147.67,41,$3.60,$4.76
40+,$38.24,13,$2.94,$3.19


## 7.Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [88]:
#Create new dataframe on Price grouped by SN.
new_group = purchase_data.groupby("SN")[["Price"]].sum()
new_group.columns = ["Total Purchase Value"]

#Create new dataframe on purchaase count grouped by SN.
new_group2 = purchase_data.groupby("SN")[["Purchase ID"]].count()
new_group2.columns = ["Purchase Count"]

#merge both new dataframes.
new_group_merge = pd.merge(new_group, new_group2, left_index = True, right_index = True, how = 'outer')

#Calculation to deduce average purchase price.
average_purchase_price_sn = new_group_merge["Total Purchase Value"]/new_group_merge["Purchase Count"]

#Add average purchase price to the dataframe.
new_group_merge["Average Purchase Price"] = average_purchase_price_sn

#Sort the dataframe in descending order, based on the total purchase value column. 
new_group_merge = new_group_merge.sort_values("Total Purchase Value", ascending = False)

#Print preview of dataframe. 
new_group_merge.head()

#formatting
new_group_merge["Total Purchase Value"] = new_group_merge["Total Purchase Value"].map("${:,.2f}".format)
new_group_merge["Average Purchase Price"] = new_group_merge["Average Purchase Price"].map("${:,.2f}".format)

#Print preview of dataframe. 
new_group_merge.head()

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


## 8.Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [114]:
#Create new dataframe with only the required columns. 
#Drop duplicate Item IDs.
#Set Item ID and Item Name as index.
new = purchase_data.drop(["Purchase ID", "SN", "Age", "Gender"], axis = 1)
new = new.drop_duplicates(subset="Item ID")
new = new.set_index(["Item ID", "Item Name"])

#Define purchase count and total purchase value through groupby function. 
purchase_count = purchase_data.groupby("Item Name")[["Item ID"]].count()
total_purchase_value_8 = purchase_data.groupby("Item ID")[["Price"]].sum()

#Merge new dataframe with total purchase value. 
#Rename columns.
new_1 = pd.merge(new, total_purchase_value_8, left_index = True, right_index = True, how = 'outer')
new_1.columns = ["Item Price", "Total Purchase Value"]

#Merge new dataframe with purchase count.
new_1 = pd.merge(new_1, purchase_count, left_index = True, right_index = True, how = 'outer')

#Rename columns. 
#Sort dataframe in descending order of purchase count.
new_1.columns = ["Item Price", "Total Purchase Value", "Purchase Count"]
new_format = new_1.sort_values("Purchase Count", ascending = False)


#formatting
new_format["Item Price"] = new_format["Item Price"].map("${:,.2f}".format)
new_format["Total Purchase Value"] = new_format["Total Purchase Value"].map("${:,.2f}".format)

#Preview dataframe.
new_format.head()

,,Item Price,Total Purchase Value,Purchase Count
Item ID,Item Name,,,
92,Final Critic,$4.88,$59.99,13
178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,$50.76,12
108,"Extraction, Quickblade Of Trembling Hands",$3.53,$31.77,9
82,Nirvana,$4.90,$44.10,9
145,Fiery Glass Crusader,$4.58,$41.22,9


## 9.Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [116]:
#Duplicate the dataframe above, rename it and sort it by descending value of total purchase value. 
new_2 = new_1.sort_values("Total Purchase Value", ascending = False)

#formatting
new_2["Item Price"] = new_2["Item Price"].map("${:,.2f}".format)
new_2["Total Purchase Value"] = new_2["Total Purchase Value"].map("${:,.2f}".format)

#Display preview of the dataframe.
new_2.head()

,,Item Price,Total Purchase Value,Purchase Count
Item ID,Item Name,,,
92,Final Critic,$4.88,$59.99,13
178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,$50.76,12
82,Nirvana,$4.90,$44.10,9
145,Fiery Glass Crusader,$4.58,$41.22,9
103,Singed Scalpel,$4.35,$34.80,8
